### Import libraries

In [1]:
import pandas as pd
import requests
import json
import csv
import numpy as np

### Load the dataset

In [2]:
sales = pd.read_csv('sales.csv')

In [3]:
sales.head(5)

,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### Find an API service

For this exercise, I'm using the [RAWG Video Games Database API](https://rawg.io/apidocs), which is the largest video game database and game discovery service. They offer various endpoints, but this time I'm focusing on their `games` endpoint to extract additional information for the games based on their title:

* Release date (`results.released`)
* Metacritic score (`results.metacritic`)

_Note:_ It is possible that not all games will have a match in the database, as searching on names isn't too fortunate because of e.g. differing capitalization, usage of special characters etc. It can also happen that not all of them will have a Metacritic score.

In [4]:
game_json = json.loads(requests.get('https://api.rawg.io/api/games').text)
game_json

{'count': 369694,
 'next': 'https://api.rawg.io/api/games?page=2',
 'previous': None,
 'results': [{'id': 3498,
   'slug': 'grand-theft-auto-v',
   'name': 'Grand Theft Auto V',
   'released': '2013-09-17',
   'tba': False,
   'background_image': 'https://media.rawg.io/media/games/b11/b115b2bc6a5957a917bc7601f4abdda2.jpg',
   'rating': 4.48,
   'rating_top': 5,
   'ratings': [{'id': 5,
     'title': 'exceptional',
     'count': 1899,
     'percent': 58.81},
    {'id': 4, 'title': 'recommended', 'count': 1086, 'percent': 33.63},
    {'id': 3, 'title': 'meh', 'count': 191, 'percent': 5.92},
    {'id': 1, 'title': 'skip', 'count': 53, 'percent': 1.64}],
   'ratings_count': 3199,
   'reviews_text_count': 17,
   'added': 10789,
   'added_by_status': {'yet': 201,
    'owned': 6899,
    'beaten': 2597,
    'toplay': 313,
    'dropped': 441,
    'playing': 338},
   'metacritic': 96,
   'playtime': 68,
   'suggestions_count': 411,
   'user_game': None,
   'reviews_count': 3229,
   'saturated_co

### Construct queries and pull information from API

* Create an array of unique games in my current dataset

In [5]:
games = sales['name'].unique()
len(games)

11325

There are 11,325 different games in my current `sales` dataset, and due to the size of this I've decided to only search for games that have at least 5MM copies sold globally (`global_sales > 5`).

In [6]:
games = sales[sales['global_sales']>5]['name'].unique()
len(games)

177

* Connect to the API based on the constracted URLs and create a dictionary

In [7]:
game_api = 'https://api.rawg.io/api/games?search='
games_dict = {}

for game in games:
    game_resp = json.loads(requests.get(game_api+game).text)['results']
    
    games_metadata_dict = {}
    games_metadata_dict.update(
        {
            'release_date': game_resp[0]['released'],
            'metacritic_score': game_resp[0]['metacritic']
        }
    )
    
    games_dict.update({game: games_metadata_dict})

In [8]:
games_dict

{'Wii Sports': {'release_date': '2006-11-19', 'metacritic_score': None},
 'Super Mario Bros.': {'release_date': '1985-09-13', 'metacritic_score': None},
 'Mario Kart Wii': {'release_date': '2008-04-27', 'metacritic_score': None},
 'Wii Sports Resort': {'release_date': '2009-07-26', 'metacritic_score': None},
 'Pokemon Red/Pokemon Blue': {'release_date': '1996-02-27',
  'metacritic_score': None},
 'Tetris': {'release_date': '1984-06-06', 'metacritic_score': None},
 'New Super Mario Bros.': {'release_date': '2006-05-15',
  'metacritic_score': None},
 'Wii Play': {'release_date': '2007-02-12', 'metacritic_score': None},
 'New Super Mario Bros. Wii': {'release_date': '2009-11-15',
  'metacritic_score': None},
 'Duck Hunt': {'release_date': '1984-04-21', 'metacritic_score': None},
 'Nintendogs': {'release_date': '2018-09-18', 'metacritic_score': None},
 'Mario Kart DS': {'release_date': '2005-11-14', 'metacritic_score': None},
 'Pokemon Gold/Pokemon Silver': {'release_date': '1999-11-21',
 

I created a dictionary with the following layout:

```
{'name': 
    {
        'release_date',
        'metacritic_score'
    }
}
```

Where `name` is the name of the game from my original `sales` dataset, and `release_date` and `metacritic_score` are coming through the API calls.

### Save the dictionary

In [9]:
np.save('games_dict.npy', games_dict)